Use the h5_import code to import the experimental data from the IR VIVO and save npy files containing the darkcount-subtracted pixel intensities (i.e., denoised images) and their respective exposure times.

Next use these images to generate radiance maps and HDR images 

In [1]:

import os
import numpy as np
import logging

 # process_and_save.py

from Step2_image_radiance_dev import load_clipped_denoised_data, process_hdr_images, save_crf_data

# Your existing setup code here
directory = './data/240329_Water_immersed'
experiment_title = 'Water_immersed'

# Load the processed data
loaded_data = load_clipped_denoised_data(directory, experiment_title)

# Process HDR images and get all necessary data
processed_data = process_hdr_images(directory, experiment_title, smoothing_lambda=1000)

# Save CRF data for future analysis
crf_file = save_crf_data(processed_data, directory, experiment_title)

print(f"CRF data saved to: {crf_file}")


Processing Water_immersed_808_BP1300
Images shape: (15, 640, 512)
Exposure times shape: (15,)
Slinear shape: (640, 512)
Sd shape: (640, 512)
bias shape: (640, 512)
Zmax_precomputed shape: (15, 640, 512)


KeyboardInterrupt: 

In [2]:
# generate_report.py

import os
import numpy as np
import logging
from Step2_image_radiance_dev import generate_multi_page_report, adaptive_weight

def load_crf_data(crf_file):
    """Load the saved CRF data."""
    print(f"Loading CRF data from: {crf_file}")
    loaded_data = np.load(crf_file, allow_pickle=True)
    
    print("Keys in the loaded file:")
    for key in loaded_data.keys():
        print(f"  {key}")
    
    processed_data = []
    i = 0
    while f'filename_{i}' in loaded_data:
        data_item = {}
        for key in loaded_data.files:
            if key.endswith(f'_{i}'):
                data_item[key[:-len(f'_{i}')]] = loaded_data[key]
        processed_data.append(data_item)
        i += 1
    
    print(f"Loaded {len(processed_data)} processed data items")
    
    if len(processed_data) == 0:
        print("Warning: No data items were loaded. Checking for alternative data structure...")
        # Check if the data was saved without indexing
        if 'intensity_samples' in loaded_data and 'log_exposures' in loaded_data and 'response_curve' in loaded_data:
            processed_data = [{
                'intensity_samples': loaded_data['intensity_samples'],
                'log_exposures': loaded_data['log_exposures'],
                'response_curve': loaded_data['response_curve'],
                'z_min': loaded_data['z_min'] if 'z_min' in loaded_data else None,
                'z_max': loaded_data['z_max'] if 'z_max' in loaded_data else None,
                'radiance_map': loaded_data['radiance_map'] if 'radiance_map' in loaded_data else None,
            }]
            print("Loaded 1 data item using alternative structure")
    
    return processed_data


# Your existing setup code here
directory = './data/240329_Water_immersed'
experiment_title = 'Water_immersed'

# Load the saved CRF data
crf_file = os.path.join(directory, "final_data", f"{experiment_title}_crf_data.npz")
processed_data = load_crf_data(crf_file)

print("Loaded processed data:")
for i, data in enumerate(processed_data):
    print(f"\nData item {i+1}:")
    for key, value in data.items():
        if isinstance(value, np.ndarray):
            print(f"  {key} shape: {value.shape}")
            if np.issubdtype(value.dtype, np.number):  # Check if the array contains numeric data
                if np.isnan(value).any() or np.isinf(value).any():
                    print(f"  Warning: {key} contains NaN or inf values")
                print(f"  {key} min: {np.min(value)}, max: {np.max(value)}")
            else:
                print(f"  {key} contains non-numeric data")
        else:
            print(f"  {key}: {value}")

# Generate the multi-page report
if processed_data:
    try:
        generate_multi_page_report(processed_data, directory, experiment_title, adaptive_weight)
        print("Report generation complete")
    except Exception as e:
        print(f"Error generating report: {str(e)}")
        import traceback
        traceback.print_exc()
else:
    print("No data to generate report. Please check the CRF data file.")


Loading CRF data from: ./data/240329_Water_immersed/final_data/Water_immersed_crf_data.npz
Keys in the loaded file:
  key_0
  radiance_map_0
  response_curve_0
  z_min_0
  z_max_0
  intensity_samples_0
  log_exposures_0
  key_1
  radiance_map_1
  response_curve_1
  z_min_1
  z_max_1
  intensity_samples_1
  log_exposures_1
  key_2
  radiance_map_2
  response_curve_2
  z_min_2
  z_max_2
  intensity_samples_2
  log_exposures_2
  key_3
  radiance_map_3
  response_curve_3
  z_min_3
  z_max_3
  intensity_samples_3
  log_exposures_3
  key_4
  radiance_map_4
  response_curve_4
  z_min_4
  z_max_4
  intensity_samples_4
  log_exposures_4
  key_5
  radiance_map_5
  response_curve_5
  z_min_5
  z_max_5
  intensity_samples_5
  log_exposures_5
  key_6
  radiance_map_6
  response_curve_6
  z_min_6
  z_max_6
  intensity_samples_6
  log_exposures_6
  key_7
  radiance_map_7
  response_curve_7
  z_min_7
  z_max_7
  intensity_samples_7
  log_exposures_7
  key_8
  radiance_map_8
  response_curve_8
  z_min_